# 全檔下載 
* [新北市路邊停車空位查詢](https://data.ntpc.gov.tw/datasets/54A507C4-C038-41B5-BF60-BBECB9D052C6/api/preview)
* [Python: zipfile 官方文件](https://docs.python.org/3/library/zipfile.html)
* [Python: Extracting a file from a zip file with a different name](https://bitdrop.st0w.com/2010/07/23/python-extracting-a-file-from-a-zip-file-with-a-different-name/)
* [Python: Tenacity (例外重作)](https://tenacity.readthedocs.io/en/latest/)

In [18]:
import time
import requests
from time import gmtime, strftime

from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen # 舊版的套件將就用

from tenacity import retry, stop_after_attempt, wait_fixed

In [2]:
url = 'http://data.ntpc.gov.tw/api/datasets/{dataSetOid}/{format}'
dataSetOid = '54A507C4-C038-41B5-BF60-BBECB9D052C6'
downloadurl = "http://data.ntpc.gov.tw/api/datasets/54A507C4-C038-41B5-BF60-BBECB9D052C6/csv/zip"

### 下載的函數，有錯會重做3次(間隔10s)

這個套件的的問題是，如果重作3遍還有exception的話，整個程式就回停下來，沒辦法忽略第三次exception。

In [24]:
@retry(stop=stop_after_attempt(3), 
       wait=wait_fixed(2))
def do_download(url, filename):
    # 下載
    resp = urlopen(url)
    zipfile = ZipFile(BytesIO(resp.read()))

    # 解壓縮 (位置與working directory相同)
    target = zipfile.infolist()[0]
    target.filename = filename
    zipfile.extract(target)
    return

In [28]:
do_download(downloadurl , 'aaa.csv')

### 這段code要開檔寫入，比較慢，也麻煩

In [ ]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
# or: requests.get(url).content

resp = urlopen(downloadurl)
zipfile = ZipFile(BytesIO(resp.read()))
filename = zipfile.namelist()[0]
for line in zipfile.open(filename).readlines():
#     print(line.decode('utf-8'))
    with open('download_test.csv', 'a', encoding='utf-8') as f:
        f.write(line.decode('utf-8'))